## Decision tree from scratch

Jacob L. Fine

March 11th, 2025


The decision tree is a way to classify data by recursively spliting data into smaller subgroups, constituting a tree. A 'split' will be defined as dividing a given set of samples into two smaller subgroups, where one of the two subgroups has all of its values for one of its features less than or equal to some threshold, and the other is greater than than that same threshold for that same feature. 

Let each node in the tree store a possible subset of the observations, and their ground truth labels. Each node will also store pointers to its two child nodes. Leaf nodes have no children, and only contain the value of the predicted class. Advantages of decisions include: (i) the complex nature of the classification process is made (often) interpretable as a series of simple decisions, (ii) they can handle well cases where the design matrix contains features that are both categorical and discrete, (iii) they capture well the non-linear nature of the relationships between features and class labels (iv) they are a non-parametric method that makes no assumptions about the distribution of the data, (v) they are fast to train and test on datasets with small/medium numbers of features and predictors.

To classify a data point, represented as a $p$-dimensional vector, we traverse the graph by comparing each feature to the feature and its threshold represented at the node. This is just like how we follow a decision tree in most contexts. We then assign the class label to the data point based on the label of the leaf node.

In the process of constructing the tree, we start with all the data at a root node, and perform the split on a given feature with a given threshold, such that the (feature, threshold) pair results in the least 'impurity' of the two child nodes (a left and right node) relative to all other possible (feature, threshold) splits. In a binary decision tree of continuous features, we use the convention that values less than the cutoff threshold are stored in the left child, whereas the remaining values are stored in teh right child. We do this recursively until the maximum depth is reached, or until all the values at a node are for the same class. 

The process of finding the best (feature, threshold) pair of values that provide the least impurity can be formalized as finding the threshold t of the jth feature that minimizes the Gini impurity, as follows:

$$\underset{(j,t)}{\text{argmin}} \space [w_{\text{left}} I(N_{\text{left}}) + w_{\text{right}} I(N_{\text{right}})]$$

Where $I(N_{\text{left}} = 1-\sum_i p_i^2)$ is the Gini impurity of the left node, based on the distribution of class labels in that node (or equivalent for right node). And $w_{\text{left}}$ is the proportion of class labels that went into the left node from the total samples. If we have $n$ samples, then $w_{\text{left}} = n_{\text{left}}/n$ is the proportion of samples that went into the left node. We may also use other measures to split the data, i.e., the information gain from the split.

We will make a decision tree from scratch in Python, and then implement it on the iris dataset from sklearn.

In [172]:
import numpy as np

# defining the node class for the decision tree
class Node:
    def __init__(self, split_feature=None,split_threshold=None,left_child=None,right_child=None,y_pred=None):
        '''
        Constructs the node.
        split_feature = the feature X_j used to split the data at that node
        split_threshold = the threshold (of one of the possible data values to split at for that feature)
        left_child = the left child of the node 
        right_child = the right child of the node
        y_pred = the predicted class of the node. Selected based on the most abundant class label in node. Only relevant if the leaf node. 
        '''
        self.split_feature = split_feature
        self.split_threshold = split_threshold
        self.left_child = left_child
        self.right_child = right_child
        self.y_pred = y_pred

# definition the actual decision tree
class DecisionTree:
    # allows for users to select the max depth like in sk learn
    def __init__(self,max_depth=int):  
        self.max_depth = max_depth
    
    def gini(self,y): # given distribution of labels at a node, get gini impurity; this is default here
        class_labels, counts = np.unique(y, return_counts=True)  # gets the class ids and their counts
        prob_y = counts/counts.sum() # normalizes the counts to get the distribution of class labels y at a node
        gini = 1-np.sum(prob_y**2) # applies gini = 1 - sum(p^2)
        return gini
    
    # Splits the data given a (feature, threshold)  pair
    def split(self,X,y,split_feature, split_threshold):
        left_row_indices = X[:,split_feature] < split_threshold # subsets the design matrix by rows less than the split threshold; row indices
        right_row_indices = ~left_row_indices  # all row indices that are not in the left split
        # returns four values: subset data matrix X corresponding and its class labels y for the split (two subsets)
        return X[left_row_indices], y[left_row_indices], X[right_row_indices], y[right_row_indices] 
    
    # finds the best (feature, threshold) pair to split data given the data at that node
    # will iterate through each feature and threshold to find the best one
    def find_best_split(self,X,y):

        best_gini = float('inf') # starts gini impurity off as infinity; then gini values will be compared to it
        best_feature, best_threshold = None, None  # will be updated until the best is found

        for feature in range(X.shape[1]): # goes through each feature
            thresholds = np.unique(X[:, feature])  # for that feature, gets all the possible data points (subset to unique cases)
            for threshold in thresholds: # iterature through the thresholds
                # calls the split function on the given feature and threshold; returns the subset data for that node
                X_left, y_left, X_right, y_right = self.split(X,y,feature,threshold)  
                if len(y_left) == 0 or len(y_right) == 0: # if there's no data in that split, keep going
                    continue
                
                gini_left = self.gini(y_left)  # given the distribtion of labels in the prospective left child node, it computes the gini impurity
                gini_right = self.gini(y_right)
                weighted_sum_gini = (len(y_left)*gini_left + len(y_right)*gini_right)/len(y)  # gets the weigthed gini impurity for that split

                if weighted_sum_gini < best_gini: # compares the computed gini to the current best gini
                    # updates the best gini, feature and threshold accordingly.
                    best_gini = weighted_sum_gini
                    best_feature = feature
                    best_threshold = threshold
                
        return best_feature, best_threshold
    
    # constructs the tree recursively by finding the best split at each node until max depth is reached; stores the current depth
    # if all the labels are the same or no split is found, it just returns the node labelled by this class
    def construct_tree(self, X, y, depth=0): 
        # first check the depth and number of unique y values stored at the node; will return labels to end the recursion process
        if len(np.unique(y))==1 or depth >= self.max_depth: # once max deprth has been reached, or there's just one unique value in the list of y values
            return Node(y_pred=np.bincount(y).argmax()) # store the class label for that node, as the most frequent count 
        
        # given the data, finds the best (split_feature, split_threshold) pair to split the data by trying all combinations of features and thresholds.
        split_feature, split_threshold = self.find_best_split(X,y)  


        if split_feature is None: # if nothing was returned for the split feature, just return the most common class
            return Node(y_pred=np.bincount(y).argmax())
        
        # uses the optimal (split_feature, split_threshold) we just found to split the data at the node
        X_left, y_left, X_right, y_right = self.split(X,y,split_feature,split_threshold)  

        # will now construct the children nodes for a node of interest based on the split done above, and recursively apply the function construct_tree to the offspring
        left_child = self.construct_tree(X_left,y_left,depth+1)   # continues recursion on left side
        right_child = self.construct_tree(X_right,y_right,depth+1)  # continues recrursion on right side

        # at this line, since we haven't already returns the leaf nodes, it will return the non-leaf nodes that have references to their children
        return Node(split_feature,split_threshold,left_child,right_child) 
    
    # given the data, an nxp matrix X, and a p-dimensional vector y, fit the model to the decision tree. Store this to the Decision tree object by giving it the attribute 'root'
    def fit(self,X,y):
        self.root = self.construct_tree(X,y) # applies the function

    # a function to classify any given one of the p-dimensional data points by traversing the tree (using recursion)
    def classify_observation(self, x, node): 
        if node.y_pred is not None: # when we reach a leaf node, y_pred is filled, so we return its value
            return node.y_pred
        
        # given the current node's split feature (which we stored before), 
        # we consider the value in x for that feature, and ask if it is less than the threshold
        if x[node.split_feature] < node.split_threshold: 
            # we call the function again on the left child since by convention, left child nodes are those with values lower than the threshold
            return self.classify_observation(x,node.left_child) 
        else:
            # otherwise, we recur to the right node
            return self.classify_observation(x,node.right_child) 
        
    # given the design matrix, actually classify each sample by applying it through the classify_observation function   
    def predict_classes(self,X):
        # iterates through each of the n observations, and traverses the decision tree with it. 
        # Will return a n-dimensional vector of predicted classes.
        return np.array([self.classify_observation(x,self.root) for x in X]) 
    
    # will recursively print results of the tree
    def display_tree(self, node=None, depth=0, feature_names=None, class_names=None, child_type="Root"):

        if node is None:
            node = self.root  # starts with the root node

        indent = "  " * depth  # indentation for readability

        if node.y_pred is not None:
            class_label = class_names[node.y_pred]  # gets the actual class name for the class y
            print(f"{indent}{child_type} -> leaf: class: {class_label}")
        else:
            feature_name = feature_names[node.split_feature]  # gets the feature class name for the feature x_j
            print(f"{indent}{child_type} -> node: {feature_name} < {node.split_threshold}")

            # recursively applies function to print children
            self.display_tree(node.left_child, depth + 1, feature_names, class_names, "left child")
            self.display_tree(node.right_child, depth + 1, feature_names, class_names, "right child")

In [173]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier, export_text
import matplotlib.pyplot as plt
from sklearn import tree

# loads iris dataset
iris = datasets.load_iris()
features = iris.feature_names
df = pd.DataFrame(iris.data, columns=features)
df['species'] = [iris.target_names[value] for value in iris.target]

# prepares data
X = iris.data  # features
y = iris.target  # labels

df['y'] = list(y)  # also represents each species class as a number

In [174]:
from sklearn.model_selection import train_test_split

# Let's now implement our model with a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)

# creating the DT object
dt = DecisionTree(max_depth=3)

# gits the model to our train and test data
dt.fit(X_train, y_train)


In [175]:
# we can see the splits our model learned in the training process (the feature, threshold pairs at non-leaf nodes)
dt.display_tree(feature_names=features, class_names=iris.target_names)

Root -> node: petal length (cm) < 3.0
  left child -> leaf: class: setosa
  right child -> node: petal width (cm) < 1.8
    left child -> node: petal length (cm) < 5.6
      left child -> leaf: class: versicolor
      right child -> leaf: class: virginica
    right child -> node: petal length (cm) < 4.9
      left child -> leaf: class: virginica
      right child -> leaf: class: virginica


In [176]:
# obtained the predicted classes, now for the TEST data (based on the train data)
y_pred_values = list(dt.predict_classes(X_test))

In [177]:
# now, we can get the accuracy, precision and recall
from sklearn.metrics import accuracy_score

# Calculates the accuracy
accuracy = round(accuracy_score(y_test, y_pred_values),4)

print('accuracy =', accuracy)

accuracy = 0.9833


This is a great evaluation! We get a ~98% accuracy for the classification task, for the test data in our model. We did have a small dataset with features that seemed to provide discriminability between the classes. The next step after a decision tree is the Random Forest - which is an ensemble of decision trees. Here, we take random samples of observations (each observation is still a $p$-dimensional vector) and train a different decision tree on each one. We classify a data point based on the majority vote of the ensemble of decision trees. The number of trees in the ensemble is a hyperparameter, and is therefore chosen in advance of running the model.